# Desafio do Titanic

O problema do Titanic é um clássico em ciência de dados, com o objetivo de prever a sobrevivência de passageiros em um naufrágio. É um problema de classificação binária baseado em um conjunto de dados com informações de 891 passageiros, onde 342 sobreviveram.

A base está disponível no github da Suzana Viana Mota, o link é https://github.com/suzanasvm/TitanicUdacity contei a base utilizada de 59.8 KB.

As atribuições da base são: PassengerId, que é a identificação dos passageiros; Survived, que indica se o passageiro sobreviveu (1) ou não (0); Pclass, que representa a classe; Name, que é o nome dos passageiros; Sex, que é o gênero; Age, que é a idade em anos; SibSp, que representa a quantidade de irmãos ou cônjuges a bordo do Titanic; Parch, que representa a quantidade de pais ou crianças a bordo do Titanic; Ticket, que é o número do bilhete; Fare, que é a tarifa paga; Cabin, que é a cabine; e Embarked, que indica o local do embarque.

In [51]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\marya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas  as pd

In [3]:
base_original = pd.read_csv("titanic-data-6.csv")

In [4]:
base_original[["nome","sobrenome"]] = base_original["Name"].str.split(",", expand=True)

In [5]:
base_original["sobrenome"].str.split(".", expand=True)

,0,1,2
0,Mr,Owen Harris,None
1,Mrs,John Bradley (Florence Briggs Thayer),None
2,Miss,Laina,None
3,Mrs,Jacques Heath (Lily May Peel),None
4,Mr,William Henry,None
...,...,...,...
886,Rev,Juozas,None
887,Miss,Margaret Edith,None
888,Miss,"Catherine Helen ""Carrie""",None
889,Mr,Karl Howell,None


In [6]:
base_original[["titulos","sobrenome", "sujeira"]] = base_original["sobrenome"].str.split(".",    expand=True)

In [7]:
base_original.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'nome', 'sobrenome',
       'titulos', 'sujeira'],
      dtype='object')

In [8]:
base_analisada = base_original[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked',
       'titulos']]


In [9]:
base_analisada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
 11  titulos      891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
base_analisada["Ticket"].dtypes == "O"

True

In [11]:
base_analisada_str = pd.DataFrame()
base_analisada_int = pd.DataFrame()

In [12]:
for i in base_analisada.columns:
    if base_analisada[i].dtypes == "O":
        base_analisada_str[i] = base_analisada[i]
    else:
        base_analisada_int[i] = base_analisada[i]


In [13]:
base_analisada_str

,Sex,Ticket,Cabin,Embarked,titulos
0,male,A/5 21171,NaN,S,Mr
1,female,PC 17599,C85,C,Mrs
2,female,STON/O2. 3101282,NaN,S,Miss
3,female,113803,C123,S,Mrs
4,male,373450,NaN,S,Mr
...,...,...,...,...,...
886,male,211536,NaN,S,Rev
887,female,112053,B42,S,Miss
888,female,W./C. 6607,NaN,S,Miss
889,male,111369,C148,C,Mr


In [17]:
base_analisada.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
base_analisada.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'titulos'],
      dtype='object')

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np

X = base_analisada[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'titulos']].values
y = base_analisada["Survived"].values

# Separa os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Imprime o número de amostras em cada conjunto
print(f"Treinamento: {len(X_train)}, Teste: {len(X_test)}")

Treinamento: 623, Teste: 268


In [20]:
dt_train = pd.DataFrame(X_train, columns= base_analisada.columns)

In [21]:
base_analisada.Survived.value_counts()/len(base_analisada.Survived)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [22]:
dt_train.Survived.value_counts()/len(dt_train.Survived)

0    0.629213
1    0.370787
Name: Survived, dtype: float64

# Explorar dados

1-Tratar null
2-Falata um dicionario para identificar o que é cada coluna e o valores. 


In [176]:
dt_train_Expl = dt_train.copy()

In [177]:

null_counts = dt_train_Expl.isnull().sum()
null_counts

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            124
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          484
Embarked         1
titulos          0
dtype: int64

In [178]:
dt_train_Expl["Age"] = dt_train_Expl["Age"].fillna(-1)
dt_train_Expl = dt_train_Expl.fillna("Nao tem")

In [193]:
dt_train_Expl[dt_train_Expl["Cabin"].apply(lambda x: len(x) > 5 and x != "Nao tem")].sort_values("Cabin")

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,titulos
281,680,1,1,male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C,Mr
418,873,0,1,male,33.0,0,0,695,5.0000,B51 B53 B55,S,Mr
362,790,0,1,male,46.0,0,0,PC 17593,79.2000,B82 B84,C,Mr
518,764,1,1,female,36.0,1,2,113760,120.0000,B96 B98,S,Mrs
621,436,1,1,female,14.0,1,2,113760,120.0000,B96 B98,S,Miss
358,298,0,1,female,2.0,1,2,113781,151.5500,C22 C26,S,Miss
503,499,0,1,female,25.0,1,2,113781,151.5500,C22 C26,S,Mrs
179,89,1,1,female,23.0,3,2,19950,263.0000,C23 C25 C27,S,Miss
240,342,1,1,female,24.0,3,2,19950,263.0000,C23 C25 C27,S,Miss
387,439,0,1,male,64.0,1,4,19950,263.0000,C23 C25 C27,S,Mr


- Retirar a coluna Cabin, pois tem poucos com valores para analisar

In [209]:
dt_train_Expl[(dt_train_Expl["Ticket"].apply(lambda x: len(x) > 7 and x != "Nao tem")) & (dt_train_Expl["Ticket"].str.contains("A"))].sort_values("Ticket")

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,titulos
310,38,0,3,male,21.0,0,0,A./5. 2152,8.0500,Nao tem,S,Mr
302,590,0,3,male,-1.0,0,0,A./5. 3235,8.0500,Nao tem,S,Mr
327,153,0,3,male,55.5,0,0,A.5. 11206,8.0500,Nao tem,S,Mr
364,489,0,3,male,30.0,0,0,A.5. 18509,8.0500,Nao tem,S,Mr
542,566,0,3,male,24.0,2,0,A/4 48871,24.1500,Nao tem,S,Mr
38,812,0,3,male,39.0,0,0,A/4 48871,24.1500,Nao tem,S,Mr
466,575,0,3,male,16.0,0,0,A/4. 20589,8.0500,Nao tem,S,Mr
55,52,0,3,male,21.0,0,0,A/4. 39886,7.8000,Nao tem,S,Mr
186,1,0,3,male,22.0,1,0,A/5 21171,7.2500,Nao tem,S,Mr
253,321,0,3,male,22.0,0,0,A/5 21172,7.2500,Nao tem,S,Mr


In [ ]:
- Retirar a coluna Ticket, pois não agrega a analise

In [98]:
# colunas categóricas
dt_train_Expl[['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'titulos']]


,Survived,Pclass,Sex,SibSp,Parch,Embarked,titulos
0,1,1,male,0,2,S,Master
1,0,3,male,0,0,S,Mr
2,1,3,female,1,1,S,Miss
3,0,2,male,1,2,S,Mr
4,0,2,male,1,1,S,Mr
...,...,...,...,...,...,...,...
618,1,3,female,0,0,S,Miss
619,0,1,male,0,0,S,Mr
620,0,3,male,2,0,S,Mr
621,1,1,female,1,2,S,Miss


In [130]:
import plotly.graph_objs as go
import plotly.subplots as sp
import pandas as pd

def analise_univariavel(df_analise,coluna_alvo):

    df1  =df_analise[["PassengerId",coluna_alvo]].groupby(coluna_alvo).count().reset_index()
    df1.insert(0,"Colunas", coluna_alvo)
    df1.columns = ["Colunas","Categoria","Qts PassengerId"]

    # Create a sample DataFrame
    df2 = df1[['Categoria','Qts PassengerId']].sort_values("Qts PassengerId", ascending=False)

    # Create a pie chart trace
    pie_trace = go.Pie(labels=df2['Categoria'], values=df2['Qts PassengerId'])

    # Create a table trace
    table_trace = go.Table(
        header=dict(values=list(df2.columns)),
        cells=dict(values=[df2[col] for col in df2.columns])
    )

    # Create a subplot with a pie chart and a table
    fig = sp.make_subplots(rows=1, cols=2, specs=[[{'type': 'pie'}, {'type': 'table'}]])

    # Add the pie chart trace to the subplot
    fig.add_trace(pie_trace, row=1, col=1)

    # Add the table trace to the subplot
    fig.add_trace(table_trace, row=1, col=2)

    # Set the layout of the subplot
    fig.update_layout(title='Análise de '+ coluna_alvo + " no Titanic" )

    # Show the plot
    return fig.show()


In [136]:
analise_univariavel(dt_train_Expl,"Embarked")

In [107]:
import plotly.express as px
df = df1
fig = px.pie(df, values="Qts PassengerId", names="Categoria", title='Population of European continent')
fig.show()


In [97]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
83,Austria,Europe,2007,79.829,8199783,36126.492700,AUT,40
119,Belgium,Europe,2007,79.441,10392226,33692.605080,BEL,56
155,Bosnia and Herzegovina,Europe,2007,74.852,4552198,7446.298803,BIH,70
191,Bulgaria,Europe,2007,73.005,7322858,10680.792820,BGR,100
383,Croatia,Europe,2007,75.748,4493312,14619.222720,HRV,191
407,Czech Republic,Europe,2007,76.486,10228744,22833.308510,CZE,203
419,Denmark,Europe,2007,78.332,5468120,35278.418740,DNK,208
527,Finland,Europe,2007,79.313,5238460,33207.084400,FIN,246
539,France,Europe,2007,80.657,61083916,30470.016700,FRA,250


In [113]:
df1

,Colunas,Categoria,Qts PassengerId
0,Pclass,1,139
1,Pclass,2,132
2,Pclass,3,352


# Reciclagem

In [ ]:
[dt_train_Expl["Cabin"][i][0] for i in range(len(dt_train_Expl["Cabin"]))]

In [71]:
import plotly.graph_objs as go
import pandas as pd

# Criar um DataFrame de exemplo com variáveis categóricas
df = pd.DataFrame({
    'Categoria': ['A', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'D'],
    'Valor': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
})

# Criar uma tabela de frequência das categorias
freq_table = df['Categoria'].value_counts()

# Criar um gráfico de pizza usando Plotly
fig = go.Figure(data=[go.Pie(labels=freq_table.index, values=freq_table.values)])

# Adicionar botões de seleção de coluna ao gráfico
buttons = []
for coluna in df['Categoria'].unique():
    button = dict(
        label=coluna,
        method="update",
        args=[{"values": [freq_table[coluna]]}]
    )
    buttons.append(button)

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            showactive=True,
            direction="down",
            x=0.5,
            y=1.2
        )
    ]
)

# Mostrar o gráfico
fig.show()



